In [ ]:
!pip install pretrainedmodels

In [ ]:
model_name = 'se_resnext101_32x4d'
approach_name = 'preproc_ext_data_thres'
use_external_data = True
use_weighted_loss = False
filename = "{}_{}".format(model_name, approach_name)
filename
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.vision import *
from fastai.metrics import error_rate

import pandas as pd
import seaborn as sns
import pretrainedmodels

print(pretrainedmodels.model_names)
batch_size = 16

# Trim black borders

In [ ]:
# # Install if needed (usually already present on Kaggle)
# # !pip install pillow numpy tqdm

# import os
# from PIL import Image
# import numpy as np
# from tqdm import tqdm

# def trim_black_border(img: Image.Image, fuzz: float = 0.05) -> Image.Image:
#     """
#     Crop away any border pixels whose all-channel values are within fuzz*255 of 0 (black).
#     """
#     arr = np.array(img)
#     thresh = int(fuzz * 255)
#     if arr.ndim == 3:
#         mask = np.any(arr > thresh, axis=2)
#     else:
#         mask = arr > thresh
#     coords = np.argwhere(mask)
#     if coords.size == 0:
#         return img  # image is all black
#     y0, x0 = coords.min(axis=0)
#     y1, x1 = coords.max(axis=0) + 1
#     return img.crop((x0, y0, x1, y1))

# def process_directory(
#     in_dir: str,
#     out_dir: str,
#     fuzz: float = 0.05,
#     shave: int = 7,
# ):
#     os.makedirs(out_dir, exist_ok=True)
#     for fname in tqdm(os.listdir(in_dir), desc=os.path.basename(in_dir)):
#         if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
#             continue
#         src_path = os.path.join(in_dir, fname)
#         img = Image.open(src_path)
#         orig_w, orig_h = img.size

#         # 1) trim black border
#         trimmed = trim_black_border(img, fuzz=fuzz)
#         trim_w, trim_h = trimmed.size
#         if (trim_w, trim_h) != (orig_w, orig_h):
#             print(f"Trimmed {fname}: {orig_w}×{orig_h} → {trim_w}×{trim_h}")

#         # 2) shave extra pixels
#         left, top = shave, shave
#         right, bottom = trim_w - shave, trim_h - shave
#         if right > left and bottom > top:
#             final = trimmed.crop((left, top, right, bottom))
#         else:
#             final = trimmed

#         # 3) save
#         dst_path = os.path.join(out_dir, fname)
#         final.save(dst_path, format="JPEG", quality=95)

# # paths (adjust if your folders live elsewhere)
# train_in  = "/kaggle/input/original-isic-2019/ISIC_2019_Training_Input/ISIC_2019_Training_Input"
# test_in   = "/kaggle/input/original-isic-2019/ISIC_2019_Test_Input/ISIC_2019_Test_Input"
# train_out = os.path.join("isic_preproc", "Training_Input")
# test_out  = os.path.join("isic_preproc", "Test_Input")

# # run!
# print("=== Processing TRAIN ===")
# process_directory(train_in,  train_out)
# print("=== Processing TEST ===")
# process_directory(test_in,   test_out)

# print("\nDone! Preprocessed images are in:", os.path.abspath("isic_preproc"))


In [ ]:
isic_path = '/kaggle/input/trimed-isic-2019/Training_Input'

isic_test_path = '/kaggle/input/trimed-isic-2019/Test_Input'

In [ ]:
def get_isic_df():
    df = pd.read_csv('/kaggle/input/original-isic-2019/ISIC_2019_Training_GroundTruth.csv')
    
    for label in df.columns[1:]:
        df.loc[df[label] == 1.0, 'label'] = label
        
    df.rename(columns={'image': 'name'}, inplace=True)
    df['name'] = df['name'].apply(lambda x: "{}/{}.jpg".format(isic_path,x))
    df = df[['name', 'label']]
    return df

def get_isic_test_df():
    df = pd.read_csv('/kaggle/input/original-isic-2019/ISIC_2019_Test_Metadata.csv', usecols=['image'])
    df.rename(columns={'image': 'name'}, inplace=True)
    return df



In [ ]:
ax = sns.countplot(x="label", data=get_isic_df(), order=['NV', 'MEL', 'BCC', 'BKL', 'AK', 'SCC', 'VASC', 'DF','UNK'])

get_isic_test_df()

In [ ]:
sd198_path = '/kaggle/input/sd-198'

sd198_isic_classes = {'Actinic_solar_Damage(Actinic_Keratosis)' : 'AK',
                       'Basal_Cell_Carcinoma' : 'BCC',
                       'Dermatofibroma' : 'DF',
                       "Becker's_Nevus" : 'NV',
                       'Blue_Nevus' : 'NV',
                       'Congenital_Nevus' : 'NV',
                       'Benign_Keratosis' : 'BKL', 
                       'Seborrheic_Keratosis' : 'BKL', 
                       'Solar_Lentigo' : 'BKL', 
                       'Lichen_Planus' : 'BKL',
                       'Malignant_Melanoma' : 'MEL', 
                       'Metastatic_Carcinoma' : 'MEL',
                       'Lentigo_Maligna_Melanoma' : 'MEL'}


def get_sd198_df():
    sd198_classes = !ls {sd198_path}
    sd198_df = pd.DataFrame()
    for c in sd198_classes:
        images = !find "{sd198_path}/{c}" -type f -name "*.jpg"
        df = pd.DataFrame()
        df['name'] = images
        
        if c not in sd198_isic_classes.keys():
            df['label'] = 'UNK'
            sd198_df = pd.concat([sd198_df, df])
        
    return sd198_df

get_sd198_df()

In [ ]:
mednode_path = '/kaggle/input/med-node/complete_mednode_dataset'

def get_mednode_df():
    mel = !find {mednode_path}/melanoma -type f -name '*jpg'
    nv = !find {mednode_path}/naevus -type f -name '*jpg'
    mel_df = pd.DataFrame(mel, columns =['image'])
    nv_df = pd.DataFrame(nv, columns =['image'])
    mel_df['label'] = 'MEL'
    nv_df['label'] = 'NV'
    df = pd.concat([mel_df, nv_df])
    df.rename(columns={'image': 'name'}, inplace=True)    
    return df


get_mednode_df()

In [ ]:
derm7pt_path = '/kaggle/input/derm7pt/release_v0/'

def get_derm7pt_df():
    df = pd.read_csv(derm7pt_path + 'meta/meta.csv', usecols=['derm', 'diagnosis'])
    df.rename(columns={'derm': 'name', 'diagnosis': 'label'}, inplace=True)
    
    df.loc[df['label'] == 'basal cell carcinoma', 'label'] = 'BCC'
    df.loc[df['label'] == 'lentigo', 'label'] = 'BKL'
    df.loc[df['label'] == 'seborrheic keratosis', 'label'] = 'BKL'
    df.loc[df['label'] == 'dermatofibroma', 'label'] = 'DF'
    df.loc[df['label'] == 'vascular lesion', 'label'] = 'VASC'
    df.loc[df['label'] == 'melanosis', 'label'] = 'BKL'
    
    df.loc[df['label'] == 'reed or spitz nevus', 'label'] = 'NV'
    df.loc[df['label'] == 'blue nevus', 'label'] = 'NV'
    df.loc[df['label'] == 'clark nevus', 'label'] = 'NV'
    df.loc[df['label'] == 'combined nevus', 'label'] = 'NV'
    df.loc[df['label'] == 'congenital nevus', 'label'] = 'NV'
    df.loc[df['label'] == 'dermal nevus', 'label'] = 'NV'
    df.loc[df['label'] == 'melanoma', 'label'] = 'MEL'
    df.loc[df['label'] == 'melanoma metastasis', 'label'] = 'MEL'
    df.loc[df['label'] == 'melanoma (in situ)', 'label'] = 'MEL'
    df.loc[df['label'] == 'melanoma (less than 0.76 mm)', 'label'] = 'MEL'
    df.loc[df['label'] == 'melanoma (0.76 to 1.5 mm)', 'label'] = 'MEL'
    df.loc[df['label'] == 'melanoma (more than 1.5 mm)', 'label'] = 'MEL'
    
    # none of the others
    df.loc[df['label'] == 'miscellaneous', 'label'] = 'UNK'
    df.loc[df['label'] == 'recurrent nevus', 'label'] = 'UNK'
    
    df = df[['name', 'label']]
    df['name'] = df['name'].apply(lambda x: "{}images/{}".format(derm7pt_path,x))
    return df

get_derm7pt_df()


In [ ]:
ph2_path = '/kaggle/input/ph2-dataset/PH2Dataset/'

def get_ph2_df():
    df = pd.read_csv(ph2_path + 'PH2_dataset.txt', sep="\|\|", skipfooter=25, engine='python', usecols=[1,3])
    df.rename(columns={'   Name ': 'name', ' Clinical Diagnosis ': 'label'}, inplace=True)
    # 0: Common Nevus, 1: Atypical Nevus, 2: Melanoma.

    df['label'] = df['label'].astype(object)
    
    df.loc[df['label'] == 0, 'label'] = 'NV'
    df.loc[df['label'] == 1, 'label'] = 'NV'
    df.loc[df['label'] == 2, 'label'] = 'MEL'
    df['name'] = df['name'].apply(lambda x: "{0}PH2_Dataset_images/{1}/{1}_Dermoscopic_Image/{1}.bmp".format(ph2_path,x.strip()))
    return df


get_ph2_df()

In [ ]:
def get_external_df():
    return pd.concat([get_sd198_df(), get_ph2_df(), get_derm7pt_df(), get_mednode_df()], sort=False)

data_df = pd.concat([get_external_df(), get_isic_df()], sort=False) if use_external_data else get_isic_df()


ax = sns.countplot(x="label", data=data_df, order=['NV', 'MEL', 'BCC', 'BKL', 'AK', 'SCC', 'DF', 'VASC', 'UNK'])

data_df

# in your Kaggle notebook (any cell, before you call detect.py)
img_list = data_df['name'].tolist()
with open('/kaggle/working/img_paths.txt', 'w') as f:
    f.write('\n'.join(img_list))
print(f"Wrote {len(img_list)} image paths to /kaggle/working/img_paths.txt")

In [ ]:
# # in your Kaggle notebook (any cell, before you call detect.py)
# img_list = data_df['name'].tolist()
# with open('/kaggle/working/img_paths.txt', 'w') as f:
#     f.write('\n'.join(img_list))
# print(f"Wrote {len(img_list)} image paths to /kaggle/working/img_paths.txt")

In [ ]:
# # from the root of your notebook (make sure yolov5/ is present)
# !git clone https://github.com/ultralytics/yolov5.git
# %cd yolov5

# # # 2. Install dependencies
# !pip install -r requirements.txt

# !python /kaggle/working/yolov5/detect.py \
#   --weights /kaggle/input/yolo_skin_lesion_detection/pytorch/default/1/best.pt \
#   --source /kaggle/working/img_paths.txt \
#   --conf 0.25 \
#   --save-txt \
#   --save-conf \
#   --save-crop \
#   --project /kaggle/working/runs/detect \
#   --name combined_run \
#   --exist-ok
